In [1]:
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline
import cv2
import pandas as pd
import numpy as np
import os

%reload_ext autoreload
%autoreload 2

import keras
from keras import backend as K
import gc
from keras.optimizers import Adam, RMSprop

from utils import predict_test, evaluate, ensemble, ThreadsafeIter, classification_predict_test
from datasets.generators import SegmentationDataGenerator, ClassificationDataGenerator

from params import args
from callbacks import get_callback
from augmentations import get_augmentations

from models.models import get_model

from losses import *

Using TensorFlow backend.


In [2]:
DATA_ROOT = '/home/p/babakhin/Branding/salt/data/'
train = pd.read_csv(os.path.join(DATA_ROOT,'train_proc_v2.csv'))
test = pd.read_csv(os.path.join(DATA_ROOT,'sample_submission.csv'))

In [ ]:
for network in args.network.split(','):
    folds = [int(f) for f in args.fold.split(',')]
    for fold in folds:
        K.clear_session()
        print('***************************** FOLD {} *****************************'.format(fold))

        MODEL_PATH = os.path.join(args.models_dir,network+args.alias)

        if fold == 0:
            if os.path.isdir(MODEL_PATH):
                raise ValueError('Such Model already exists')
            os.system("mkdir {}".format(MODEL_PATH))
            os.system("cp params.py {}".format(MODEL_PATH))

        df_train = train[train.fold != fold].copy().reset_index(drop=True)
        df_valid = train[train.fold == fold].copy().reset_index(drop=True)

        ids_train, ids_valid = df_train[df_train.unique_pixels>1].id.values, df_valid.id.values

        # Fold 0
        # Training on 3127 samples
        # Validating on 810 samples
        # ids_train, ids_valid = df_train[(df_train.unique_pixels>1)&(~df_train.id.isin(bad_masks))].id.values, df_valid.id.values


        print('Training on {} samples'.format(ids_train.shape[0]))
        print('Validating on {} samples'.format(ids_valid.shape[0]))

        # Initialize Model
        weights_path = os.path.join(MODEL_PATH,'fold_{fold}.hdf5'.format(fold=fold))

        print(weights_path.split('/')[-2:])


        model, preprocess = get_model(network, input_shape=(args.input_size, args.input_size, 3), train_base=True)
        print(model.summary())
        model.compile(optimizer=RMSprop(lr=args.learning_rate), loss=make_loss(args.loss_function),
                          metrics=[dice_coef, jacard_coef])

        if args.weights is None:
            print('No weights passed, training from scratch')
        else:
            wp = args.weights.format(fold)
            print('Loading weights from {}'.format(wp))
            model.load_weights(wp, by_name=True)

        augs = get_augmentations(args.augmentation_name, p=args.augmentation_prob, input_shape=(args.input_size, args.input_size, 3))


        dg = SegmentationDataGenerator(input_shape=(args.input_size, args.input_size), batch_size = args.batch_size, augs = augs,
                          preprocess = preprocess)

        train_generator = dg.train_batch_generator(ids_train)
        validation_generator = dg.evaluation_batch_generator(ids_valid)

        callbacks = get_callback(args.callback, weights_path=weights_path,
                                fold = fold)

        # Fit the model with Generators:
        model.fit_generator(generator=ThreadsafeIter(train_generator),
                        steps_per_epoch=ids_train.shape[0] // args.batch_size * 2,
                        epochs=args.epochs,
                        callbacks=callbacks,
                        validation_data=ThreadsafeIter(validation_generator),
                        validation_steps=np.ceil(ids_valid.shape[0] / args.batch_size),
                        workers=8)

        model.load_weights(weights_path)

        # SAVE OOF PREDICTIONS
        dir_path = os.path.join(MODEL_PATH,'oof')
        os.system("mkdir {}".format(dir_path))
        pred = predict_test(model=model,
                        preds_path=dir_path,
                        oof=True,
                        ids=ids_valid,
                        batch_size=args.batch_size*4,
                        thr=0.5,
                        TTA='',
                        preprocess=preprocess)

        # SAVE TEST PREDICTIONS
#         dir_path = os.path.join(MODEL_PATH,'fold_{}'.format(fold))
#         os.system("mkdir {}".format(dir_path))
#         pred = predict_test(model=model,
#                         preds_path=dir_path,
#                         oof=False,
#                         ids=test.id.values,
#                         batch_size=args.batch_size*2,
#                         thr=0.5,
#                         TTA='flip',
#                         preprocess=preprocess)

        K.clear_session()
        # Run a single fold
        # break

***************************** FOLD 0 *****************************
Training on 3127 samples
Validating on 810 samples
['unet_resnet_34_exp_2_128_optical_distortion', 'fold_0.hdf5']
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 128, 128, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 134, 134, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)             

Epoch 1/300
194/194 [==============================] - 47s 241ms/step - loss: 0.6252 - dice_coef: 0.4643 - jacard_coef: 0.3066 - val_loss: 0.4897 - val_dice_coef: 0.6304 - val_jacard_coef: 0.4620
Epoch 2/300
194/194 [==============================] - 40s 205ms/step - loss: 0.5038 - dice_coef: 0.5536 - jacard_coef: 0.3867 - val_loss: 0.3232 - val_dice_coef: 0.7507 - val_jacard_coef: 0.6041
Epoch 3/300
194/194 [==============================] - 40s 206ms/step - loss: 0.4640 - dice_coef: 0.5906 - jacard_coef: 0.4227 - val_loss: 0.3003 - val_dice_coef: 0.7766 - val_jacard_coef: 0.6381
Epoch 4/300
194/194 [==============================] - 40s 206ms/step - loss: 0.4392 - dice_coef: 0.6136 - jacard_coef: 0.4472 - val_loss: 0.2921 - val_dice_coef: 0.8021 - val_jacard_coef: 0.6715
Epoch 5/300
194/194 [==============================] - 40s 206ms/step - loss: 0.4280 - dice_coef: 0.6289 - jacard_coef: 0.4640 - val_loss: 0.2399 - val_dice_coef: 0.8222 - val_jacard_coef: 0.7012
Epoch 6/300
194/194 

Epoch 42/300
194/194 [==============================] - 40s 206ms/step - loss: 0.3170 - dice_coef: 0.7364 - jacard_coef: 0.5899 - val_loss: 0.1602 - val_dice_coef: 0.9065 - val_jacard_coef: 0.8313
Epoch 43/300
194/194 [==============================] - 40s 206ms/step - loss: 0.3257 - dice_coef: 0.7327 - jacard_coef: 0.5858 - val_loss: 0.1609 - val_dice_coef: 0.9082 - val_jacard_coef: 0.8343
Epoch 44/300
194/194 [==============================] - 40s 206ms/step - loss: 0.3190 - dice_coef: 0.7408 - jacard_coef: 0.5944 - val_loss: 0.1620 - val_dice_coef: 0.9061 - val_jacard_coef: 0.8306

Epoch 00044: ReduceLROnPlateau reducing learning rate to 1.56249996052793e-06.



***************************** FOLD 1 *****************************
Training on 3134 samples
Validating on 804 samples
['unet_resnet_34_exp_2_128_optical_distortion', 'fold_1.hdf5']
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 128, 128, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 134, 134, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)            

Epoch 1/300
194/194 [==============================] - 45s 233ms/step - loss: 0.5809 - dice_coef: 0.4666 - jacard_coef: 0.3092 - val_loss: 0.3956 - val_dice_coef: 0.6453 - val_jacard_coef: 0.4811
Epoch 2/300
194/194 [==============================] - 40s 205ms/step - loss: 0.4844 - dice_coef: 0.5771 - jacard_coef: 0.4098 - val_loss: 0.2910 - val_dice_coef: 0.7852 - val_jacard_coef: 0.6510
Epoch 3/300
194/194 [==============================] - 40s 205ms/step - loss: 0.4371 - dice_coef: 0.6236 - jacard_coef: 0.4579 - val_loss: 0.2823 - val_dice_coef: 0.7961 - val_jacard_coef: 0.6662
Epoch 4/300
194/194 [==============================] - 40s 205ms/step - loss: 0.4221 - dice_coef: 0.6430 - jacard_coef: 0.4785 - val_loss: 0.2560 - val_dice_coef: 0.8287 - val_jacard_coef: 0.7128
Epoch 5/300
194/194 [==============================] - 40s 205ms/step - loss: 0.4043 - dice_coef: 0.6591 - jacard_coef: 0.4967 - val_loss: 0.2603 - val_dice_coef: 0.8241 - val_jacard_coef: 0.7078
Epoch 6/300
194/194 

In [ ]:
EXPEREMNT with models without folds!!

In [ ]:
resnet_50_224: 0.79148 / 0.84983 / 0.82436
resnet50_fpn_old (~0.1600): 0.82296 / 0.87922 / 0.85315
  
resnet34_new: 0.80605 / 0.86341 / 0.83653
    CHECK WEIGHTS TIME
    DELETE PREPROCESSING
    
OLD_UPDATED_WITH_PARAMS_BATCH_SIZE_AND_CROP (0.1900): 0.80395 / 0.8557 / 0.82988
Return Batch Size and Crop (~0.1541): 0.81444 / 0.86188 / 0.83796
Set Batch size to 32 (~0.1514): 0.81938 / 0.86952 / 0.84411
PAD_AND_CROP:
    
Freezing beforehand (0.1762)

In [ ]:
args.early_stop_patience

In [ ]:
MODEL_PATH

In [ ]:
MODEL_PATH = '/home/branding_images/salt/unet_resnet_34_exp_2_128_shift_p_1_again/'

In [ ]:
'/home/branding_images/salt/classification/oof'

In [ ]:
res = evaluate([MODEL_PATH], train[train.fold.isin([0,1])].id.values, 0.5, classification='')
print("{} / {} / {}".format(np.round(np.mean(res['iout']),5),np.round(np.mean(res['dice']),5),np.round(np.mean(res['jacard']),5)))

In [ ]:
unet_resnet_34_exp_2_128
0.83067 / 0.87668 / 0.85237
unet_resnet_34_exp_2_128_again
0.82038 / 0.87012 / 0.84568

unet_resnet_34_exp_2_128_shift_p_1
0.81983 / 0.86893 / 0.84417
unet_resnet_34_exp_2_128_shift_p_1_again
0.82187 / 0.86955 / 0.84518


unet_resnet_34_exp_2_128_shift_p_08_shift_025
0.82429 / 0.87048 / 0.84619

unet_resnet_34_exp_2_128_shift_p_07_scale_02
0.81884 / 0.86482 / 0.84085

unet_resnet_34_exp_2_128_shift_p_07_scale_06
0.82776 / 0.87532 / 0.85079
unet_resnet_34_exp_2_128_shift_p_07_scale_06_again
0.82212 / 0.8738 / 0.84824

In [ ]:
initial 2 folds 0.80812 / 0.86215 / 0.83612
again: 0.81995 / 0.86938 / 0.84416
    
unet_resnet_34_exp_2_folds_lr_001:
0.76258 / 0.81572 / 0.78989

unet_resnet_34_exp_2_folds_lr_0001:
0.78885 / 0.84258 / 0.81624

unet_resnet_34_exp_2_folds_vertical_flip
0.7847 / 0.83687 / 0.81089

unet_resnet_34_exp_2_folds_transpose_corr_v2
0.81183 / 0.8625 / 0.83739
unet_resnet_34_exp_2_folds_transpose_corr_v2_again
0.79981 / 0.85433 / 0.82802

unet_resnet_34_exp_2_folds_contrast (0.2-0.1)
0.8109 / 0.86076 / 0.83574
unet_resnet_34_exp_2_folds_contrast_again
0.81629 / 0.86572 / 0.84094

unet_resnet_34_exp_2_folds_contrast_corr (0.1-0.1)
0.80496 / 0.85516 / 0.82939
unet_resnet_34_exp_2_folds_contrast_corr_again
0.81152 / 0.8637 / 0.837

unet_resnet_34_exp_2_folds_blur
0.79461 / 0.84789 / 0.82089

unet_resnet_34_exp_2_folds_gaus_noise
0.80719 / 0.8622 / 0.83523
unet_resnet_34_exp_2_folds_gaus_noise_again
0.79678 / 0.84703 / 0.8213

unet_resnet_34_exp_2_folds_no_contrast_brightness
0.81221 / 0.8642 / 0.83845
unet_resnet_34_exp_2_folds_no_contrast_brightness_again
0.80657 / 0.85867 / 0.83347

unet_resnet_34_exp_2_folds_one_of_contrast_brightness:
0.77881 / 0.83784 / 0.81064
unet_resnet_34_exp_2_folds_one_of_contrast_brightness_again
0.81196 / 0.86585 / 0.84046

unet_resnet_34_exp_2_folds_shift_limit_1625
0.8176 / 0.86312 / 0.83837
unet_resnet_34_exp_2_folds_shift_limit_1625_again
0.82224 / 0.8723 / 0.84684

unet_resnet_34_exp_2_folds_brightness_contrast_0_5
0.81481 / 0.8672 / 0.84179

unet_resnet_34_better_augs_224
0.82739 / 0.87377 / 0.84944

In [ ]:
res = evaluate([MODEL_PATH], train.id.values, 0.5, classification='')
print("{} / {} / {}".format(np.round(np.mean(res['iout']),5),np.round(np.mean(res['dice']),5),np.round(np.mean(res['jacard']),5)))

In [ ]:
models = ['resnet50_fpn_pad_crop',
'resnet50_fpn_freeze_for_5_epochs',
'resnet50_fpn_hor_flip_aug',
'resnet_50_224',
'resnet50_fpn_128',
'resnet101_fpn_96_bs_32',
'resnet50_fpn_96_soft_early_stopping',
'resnet50_fpn_jacard_only',
'unet_128_96',
'resnet50_fpn_spatial_do',
'resnet34_fpn',
'resnet50_fpn_old',
'unet_mobilenet_128']
model_pathes = ['/home/branding_images/salt/'+x for x in models]

In [ ]:


# import numpy as np
# import pydensecrf.densecrf as dcrf
# from skimage.io import imread, imsave
# from pydensecrf.utils import unary_from_labels, create_pairwise_bilateral
# from skimage.color import gray2rgb
# from skimage.color import rgb2gray
# import matplotlib.pyplot as plt
# import pandas as pd
# from tqdm import tqdm
# %matplotlib inline



In [ ]:
train['iout'] = res['iout']
train.groupby('fold').iout.aggregate('mean')

In [ ]:
MODEL_PATH

In [ ]:
#pred = ensemble(model_pathes,[0,1,2,3,4],test.id.values,0.5)
pred = ensemble([MODEL_PATH],[0,1,2,3,4],test.id.values,0.5, classification='')

In [ ]:
pred

In [ ]:
pred

In [ ]:
test['rle_mask'] = pred
test[['id','rle_mask']].to_csv('unet_resnet_34_better_augs_224_tta_5_folds_83533.csv',index=False)

In [ ]:
def plot2x2Array(image, mask):
    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(image)
    axarr[1].imshow(mask)
    axarr[0].grid()
    axarr[1].grid()
    axarr[0].set_title('Image')
    axarr[1].set_title('Mask')
    
for i in range(5):
    image, mask = dataset[np.random.randint(0, len(dataset))]
    plot2x2Array(image, mask)

In [ ]:
def show_results(idx):
    from rle import rle_decode
    img = rle_decode(pred[idx],(101,101))
    plt.figure(figsize=(4,4))
    plt.imshow(img)
    
    plt.figure(figsize=(4,4))
    plt.imshow(cv2.imread('train/images/{}.png'.format(ids_valid[idx])))
    plt.figure(figsize=(4,4))
    plt.imshow(cv2.imread('train/masks/{}.png'.format(ids_valid[idx])))
    

In [ ]:
MODEL_PATH

In [ ]:
MODEL_PATH = os.path.join(args.models_dir,network+args.alias)

In [ ]:
for i in range(10):
    show_results(i)

# Classification

In [ ]:
validation_probs=[]
network = 'classification'
folds = [int(f) for f in args.fold.split(',')]
for fold in folds:
    K.clear_session()
    print('***************************** FOLD {} *****************************'.format(fold))

    MODEL_PATH = os.path.join(args.models_dir,network+args.alias)

    if fold == 0:
        if os.path.isdir(MODEL_PATH):
            raise ValueError('Such Model already exists')
        os.system("mkdir {}".format(MODEL_PATH))
        os.system("cp params.py {}".format(MODEL_PATH))

    df_train = train[train.fold != fold].copy().reset_index(drop=True)
    df_valid = train[train.fold == fold].copy().reset_index(drop=True)

    ids_train, ids_valid = df_train.id.values, df_valid.id.values

    print('Training on {} samples'.format(ids_train.shape[0]))
    print('Validating on {} samples'.format(ids_valid.shape[0]))

    # Initialize Model
    weights_path = os.path.join(MODEL_PATH,'fold_{fold}.hdf5'.format(fold=fold))

    print(weights_path.split('/')[-2:])

    model, preprocess = get_model(network, input_shape=(args.input_size, args.input_size, 3), train_base=True)
    print(model.summary())
    model.compile(optimizer=RMSprop(lr=args.learning_rate), loss='binary_crossentropy',
                      metrics=['accuracy'])

    if args.weights is None:
        print('No weights passed, training from scratch')
    else:
        wp = args.weights.format(fold)
        print('Loading weights from {}'.format(wp))
        model.load_weights(wp, by_name=True)

    augs = get_augmentations(args.augmentation_name, p=args.augmentation_prob, input_shape=(args.input_size, args.input_size, 3))


    dg = ClassificationDataGenerator(input_shape=(args.input_size, args.input_size), batch_size = args.batch_size, augs = augs,
                      preprocess = preprocess)

    train_generator = dg.train_batch_generator(ids_train)
    validation_generator = dg.evaluation_batch_generator(ids_valid)

    callbacks = get_callback(args.callback, weights_path=weights_path,
                            fold = fold)

    # Fit the model with Generators:
    model.fit_generator(generator=ThreadsafeIter(train_generator),
                    steps_per_epoch=ids_train.shape[0] // args.batch_size * 2,
                    epochs=args.epochs,
                    callbacks=callbacks,
                    validation_data=ThreadsafeIter(validation_generator),
                    validation_steps=np.ceil(ids_valid.shape[0] / args.batch_size),
                    workers=8)

    model.load_weights(weights_path)

    # SAVE OOF PREDICTIONS
    dir_path = os.path.join(MODEL_PATH,'oof')
    os.system("mkdir {}".format(dir_path))
    pred = classification_predict_test(model=model,
                    preds_path=dir_path,
                    oof=True,
                    ids=ids_valid,
                    batch_size=args.batch_size*4,
                    thr=0.5,
                    TTA='',
                    preprocess=preprocess)
    df_valid['prob'] = pred
    validation_probs.append(df_valid)

    # SAVE TEST PREDICTIONS
    dir_path = os.path.join(MODEL_PATH,'fold_{}'.format(fold))
    os.system("mkdir {}".format(dir_path))
    pred = classification_predict_test(model=model,
                    preds_path=dir_path,
                    oof=False,
                    ids=test.id.values,
                    batch_size=args.batch_size*4,
                    thr=0.5,
                    TTA='',
                    preprocess=preprocess)
    test['prob'] = pred
    test.to_csv(os.path.join(dir_path,'probs_test_fold_{}.csv'.format(fold)),index=False)

    K.clear_session()
    # Run a single fold
    # break

dir_path = os.path.join(MODEL_PATH,'oof')
pd.concat(validation_probs).to_csv(os.path.join(dir_path,'probs_oof.csv'),index=False)

In [ ]:
dir_path = os.path.join(MODEL_PATH,'oof')
tt = pd.read_csv(os.path.join(dir_path,'probs_oof_fold_{}.csv'.format(fold)))
tt[pd.isnull(tt['rle_mask'])&(tt.unique_pixels > 1)].prob.describe()

In [ ]:
tt.head(30)

In [ ]:
Add random crops augmentation

Do not load the weights (start training from scratch?)

Apply Cyclic LR

Finetune models for each fold (maybe stucked in local optima). Probably, finetune without augmentations

TRY TTA and Blend of top-5 models.

Add more augmentations to 'initial'

Delete VGG from unet_resnet_50 in the end
Compare keras resnet_50 and resnet_50_fixed. There is average pooling at the end.
And no padding in the beginning. And min sizes are different

I think this pipeline works really well considering its simplicity, but I made a big mistake.
- I didn't noticed that some test outputs from stage 1 were horribly wrong until today and didn't
have chance to correct it. :( I should have found the issue early only if
I had good visualization or simple sanity check logic.

The best lesson from this competition is that good visualization/analyzing tool is really important.
                             
Random crops. E.g. the same 96 and resize during inference.
    While random crops 96 in the training

In [ ]:
UNET d4 augs (0.62)
! LINKNET (0.70)
cv.gaussian blur for masks predicted smooting

loss: iou is overprediction. bce+jaccard
we could use 96 or 128 size.
bce+jaccard: 0.15 on validation; iou -- 0.8. LB 0.74

Start with predicting: whether an image has mask at all -- binary classification.
    Then multiply these prior probabilities on mask obtained! Again: classification pipeline.
        
Some 100% incorrect masks!

FIND DUPLICATES OF IMAGES! In train and test

Change Kaggle architectures with comments from ods:
Use SpatialDropout2D and decrease dropout rate
Conv2DTranspose works worse than NN upsampling + conv
"You don't need multiprocessing, it will run batches preparation in different processes"
-- use multiprocessing in keras

CRF postprocessing for predictions obtained! Just Function from kaggle -- you could check on validation!

TRY TO BINIRIZE MASK AFTER APPLYING RESIZE
HOW IS MY DICE CALCULATED? IT SHOULD HAVE THRESHOLD

Increase Smoothing Parameter in Dice!

Write Own Cyclic LR? Custom with saving checkpoints!

Pretrain model on classification (whether the mask exist)

In [ ]:
Try to use own reduceLR. Because it's not clear where keras starts with decreasing LR
(I guess, from the last epoch. Not the best! So, reinitialize the model. Add some functions to not repeat the code)

Pretrained model for each fold:
    pretrain of pseudolabels (the best ensemble so far) and high augmentations
    or train simultaneously
    
Try different LR and policies (factor)

Jacard loss with threshold
https://github.com/lyakaap/Kaggle-Carvana-3rd-Place-Solution/blob/master/losses.py
    
Evgeny used Adam with inital LR=1e-4 and several lr drops at fixed steps.

Segnet and
Linknet (https://github.com/davidtvs/Keras-LinkNet/tree/master/models) architectures
"""
model = LinkNet(num_classes, input_shape=input_shape)
        model = model.get_model(
            pretrained_encoder=pretrained_encoder, weights_path=weights_path
        )
"""

DeepLab v3+
https://github.com/bonlime/keras-deeplab-v3-plus

Tensorboard callback with images!
https://github.com/davidtvs/Keras-LinkNet/blob/master/callbacks.py
https://stackoverflow.com/questions/43784921/how-to-display-custom-images-in-tensorboard-using-keras